In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import csv
import os
from datetime import date, timedelta, datetime
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Table 1

In [1]:
demo_df = pd.read_csv('./demographic_data_summary_final.csv')
demo_df.head(3)

In [9]:
adh_dir = './cgm_count_by_day/'
adh_lst = []

for filename in os.listdir(adh_dir):
  if filename[-4:] == '.csv':
    df = pd.read_csv(adh_dir + filename)
    valid = list(df.Valid.values)
    short_valid = valid[:30].count(1) / 30
    long_valid = valid.count(1) / len(valid)
    valid = 1 if long_valid >= .7 else 0 # long term adherence is valid if long_valid > .7
    adh_lst.append([filename[:-4], short_valid, long_valid, valid])
    # break

In [10]:
valid_num = [i[3] for i in adh_lst].count(1)
invalid_num = [i[3] for i in adh_lst].count(0)
print(valid_num, invalid_num)

57 51


In [11]:
df_adh = pd.DataFrame(adh_lst, columns=['app_id', 'short_valid', 'long_valid', 'adherence'])

In [2]:
df_merge = pd.merge(demo_df, df_adh, on="app_id")
df_merge.head(3)

In [13]:
gender = []
for i in df_merge.gender.values:
  if i == '1: Male':
    gender.append(1)
  elif i == '2: Female':
    gender.append(2)
  elif i == '3: Other':
    gender.append(3)
  else:
    gender.append(0)

race = []
for i in df_merge.race.values:
  if i == '1: White':
    race.append(1)
  elif i == '2: Black':
    race.append(2)
  elif i == '3: Others':
    race.append(3)
  else:
    race.append(0)

ins_type = []
for i in df_merge.insulin_type.values:
  if i == 'Insulin pump':
    ins_type.append(1)
  elif i == 'Multiple daily injections':
    ins_type.append(2)
  else:
    ins_type.append(0)

In [3]:
df_merge.insert(3, 'Gender', gender)
df_merge.insert(7, 'Race', race)
df_merge.insert(10, 'Insulin_type', ins_type)
df_merge.head()

In [4]:
new_df = df_merge.drop(['app_id', 'gender', 'race', 'insulin_type', 'sweetgoals_id', 'location', 'long_valid'], axis=1)
new_df.head(3)

In [16]:
new_df.to_csv('./multi_model_predictors.csv', index=None)

In [17]:
from scipy.stats.contingency import odds_ratio
def get_odds(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases):
  res = odds_ratio([[exposed_cases, unexposed_cases], [exposed_noncases, unexposed_noncases]])
  print('odds ratio', res.statistic)
  print('95% CI', res.confidence_interval(confidence_level=0.95))

In [18]:
import scipy.stats as stats
def get_fisher_p(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases):
  table = [[exposed_cases, unexposed_cases], [exposed_noncases, unexposed_noncases]]
  _, pvalue = stats.fisher_exact(table)
  print("p-Value:", pvalue)
  # return pvalue

In [19]:
# reference
def get_unexposed(unexposed_df):
  print('unexposed:', unexposed_df.shape)
  unexposed_cases = list(unexposed_df.adherence.values).count(1) # high ahderence (what we want to find)
  unexposed_noncases = list(unexposed_df.adherence.values).count(0)
  print(unexposed_cases, unexposed_noncases)
  return unexposed_cases, unexposed_noncases

In [20]:
def get_exposed(exposed_df):
  print('exposed:', exposed_df.shape)
  exposed_cases = list(exposed_df.adherence.values).count(1)
  exposed_noncases = list(exposed_df.adherence.values).count(0)
  print(exposed_cases, exposed_noncases)
  return exposed_cases, exposed_noncases

In [21]:
age = {1:[19, 21], 2:[22, 25], 3:[26, 29]}

for i in range(2, 4):
  print('---', age[i])
  unexposed_df = new_df[new_df['age']>= 19][new_df['age']<= 21]
  exposed_df = new_df[new_df['age']>= age[i][0]][new_df['age']<= age[i][1]]
  exposed_cases, exposed_noncases = get_exposed(exposed_df)
  unexposed_cases, unexposed_noncases =  get_unexposed(unexposed_df)
  get_odds(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)
  get_fisher_p(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)

--- [22, 25]
exposed: (38, 9)
21 17
unexposed: (44, 9)
22 22
odds ratio 1.2321103887012423
95% CI ConfidenceInterval(low=0.47388598228438855, high=3.2298776416877386)
p-Value: 0.6634417371780723
--- [26, 29]
exposed: (26, 9)
14 12
unexposed: (44, 9)
22 22
odds ratio 1.1640997801075932
95% CI ConfidenceInterval(low=0.3964951907664974, high=3.454789004858414)
p-Value: 0.8080827028123225


<ipython-input-21-1e8b58370eed>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exposed_df = new_df[new_df['age']>= age[i][0]][new_df['age']<= age[i][1]]
<ipython-input-21-1e8b58370eed>:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exposed_df = new_df[new_df['age']>= age[i][0]][new_df['age']<= age[i][1]]


In [22]:
exposed_df1 = new_df[new_df['A1c']> 7.5][new_df['A1c']<= 9]
exposed_df2 = new_df[new_df['A1c']> 9]
a1c = {2: '(7.5, 9]', 3:'>9'}
exposed_ = {2:exposed_df1, 3: exposed_df2}

for i in range(2, 4):
  print('---', a1c[i])
  unexposed_df = new_df[new_df['A1c']>= 6][new_df['A1c']<= 7.5]
  exposed_df = exposed_[i]
  exposed_cases, exposed_noncases = get_exposed(exposed_df)
  unexposed_cases, unexposed_noncases =  get_unexposed(unexposed_df)
  get_odds(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)
  get_fisher_p(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)

--- (7.5, 9]
exposed: (49, 9)
24 25
unexposed: (25, 9)
17 8
odds ratio 0.4566472170537203
95% CI ConfidenceInterval(low=0.14211653373295022, high=1.3705608514078973)
p-Value: 0.1433543199654274
--- >9
exposed: (20, 9)
9 11
unexposed: (25, 9)
17 8
odds ratio 0.39364592156874756
95% CI ConfidenceInterval(low=0.09573058591567811, high=1.5228916206337009)
p-Value: 0.1417754095671262


<ipython-input-22-7dbefa329db9>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exposed_df1 = new_df[new_df['A1c']> 7.5][new_df['A1c']<= 9]
<ipython-input-22-7dbefa329db9>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unexposed_df = new_df[new_df['A1c']>= 6][new_df['A1c']<= 7.5]
<ipython-input-22-7dbefa329db9>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unexposed_df = new_df[new_df['A1c']>= 6][new_df['A1c']<= 7.5]


In [23]:
print('gender')
unexposed_df = new_df[new_df['Gender'] == 1]
exposed_df = new_df[new_df['Gender'] == 2]
exposed_cases, exposed_noncases = get_exposed(exposed_df)
unexposed_cases, unexposed_noncases =  get_unexposed(unexposed_df)
get_odds(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)
get_fisher_p(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)

gender
exposed: (78, 9)
45 33
unexposed: (26, 9)
11 15
odds ratio 1.8482698160096789
95% CI ConfidenceInterval(low=0.6911954917158486, high=5.084015873314742)
p-Value: 0.18302736441969053


In [24]:
exposed_df1 = new_df[new_df['diagnosis_duration']> 7][new_df['diagnosis_duration']<= 15]
exposed_df2 = new_df[new_df['diagnosis_duration']> 15]
a1c = {2: '(7, 15]', 3:'> 15'}
exposed_ = {2:exposed_df1, 3: exposed_df2}

for i in range(2, 4):
  print('---', a1c[i])
  unexposed_df = new_df[new_df['diagnosis_duration']>= 2][new_df['diagnosis_duration']<= 7]
  exposed_df = exposed_[i]
  exposed_cases, exposed_noncases = get_exposed(exposed_df)
  unexposed_cases, unexposed_noncases =  get_unexposed(unexposed_df)
  get_odds(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)
  get_fisher_p(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)

--- (7, 15]
exposed: (57, 9)
32 25
unexposed: (23, 9)
11 12
odds ratio 1.3905047847488285
95% CI ConfidenceInterval(low=0.47252229251588485, high=4.136974220320114)
p-Value: 0.6215056995246082
--- > 15
exposed: (26, 9)
12 14
unexposed: (23, 9)
11 12
odds ratio 0.9363474303126542
95% CI ConfidenceInterval(low=0.2632809163356799, high=3.3232931653101194)
p-Value: 1.0


<ipython-input-24-d9ad25ae8b8c>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exposed_df1 = new_df[new_df['diagnosis_duration']> 7][new_df['diagnosis_duration']<= 15]
<ipython-input-24-d9ad25ae8b8c>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unexposed_df = new_df[new_df['diagnosis_duration']>= 2][new_df['diagnosis_duration']<= 7]
<ipython-input-24-d9ad25ae8b8c>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unexposed_df = new_df[new_df['diagnosis_duration']>= 2][new_df['diagnosis_duration']<= 7]


In [25]:
print('short valid')
unexposed_df = new_df[new_df['short_valid'] >= 0.7]
exposed_df = new_df[new_df['short_valid'] < 0.7]
exposed_cases, exposed_noncases = get_exposed(exposed_df)
unexposed_cases, unexposed_noncases =  get_unexposed(unexposed_df)
get_odds(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)
get_fisher_p(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)

short valid
exposed: (38, 9)
8 30
unexposed: (70, 9)
49 21
odds ratio 0.11697509871993386
95% CI ConfidenceInterval(low=0.03937338630694812, high=0.3137802500318528)
p-Value: 1.936344468977107e-06


In [26]:
print('Insurance')
unexposed_df = new_df[new_df['Insurance_0'] == 1] # private as ref
exposed_df = new_df[new_df['Insurance_0'] == 2]
exposed_cases, exposed_noncases = get_exposed(exposed_df)
unexposed_cases, unexposed_noncases =  get_unexposed(unexposed_df)
get_odds(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)
get_fisher_p(exposed_cases, unexposed_cases, exposed_noncases, unexposed_noncases)

Insurance
exposed: (18, 9)
11 7
unexposed: (78, 9)
39 39
odds ratio 1.564104614582513
95% CI ConfidenceInterval(low=0.49311192848327007, high=5.29076205941997)
p-Value: 0.4422268845619656


# Table 2

## Demographic

In [46]:
demo_file = './demographic_data_summary_final.csv'
df = pd.read_csv(demo_file)
df.head(3)

,app_id,sweetgoals_id,gender,A1c,age,race,diagnosis_duration,insulin_type,location,Insurance_0
0,643dc3f3e796c60013672184,617,2: Female,8.1,21,1: White,10.6,Insulin pump,New Jersey,1.0
1,640a0c11649045001676e53c,565,1: Male,6.7,28,1: White,19.0,Insulin pump,New York,1.0
2,6407b513649045001676e126,540,1: Male,8.1,28,1: White,4.1,Insulin pump,Ohio,1.0


In [47]:
print('total number of subject:', df.app_id.count())

total number of subject: 108


In [48]:
race_dict = {}
race_lst = list(df.race.values)
for i in df.race.unique():
  race_dict.update({i: race_lst.count(i)})

print(race_dict)

{'1: White': 77, '3: Others': 7, nan: 12, '2: Black': 12}


In [49]:
type_dict = {}
type_lst = list(df.insulin_type.values)
for i in df.insulin_type.unique():
  type_dict.update({i: type_lst.count(i)})

print(type_dict)

{'Insulin pump': 88, 'Multiple daily injections': 20}


In [50]:
age = df.age.values[~np.isnan(df.age.values)]

print('age avg and std:', np.mean(age), '+/', np.std(age)) # only keep intergers?
print('age range:', np.min(age), '-' , np.max(age))
print('not available:', len(df.age.values[np.isnan(df.age.values)]))

age avg and std: 22.84259259259259 +/ 3.0158901303700785
age range: 19 - 29
not available: 0


In [51]:
gender_lst = []
for i in range(len(df.sweetgoals_id.values)):
  gender_lst.append(df.gender.values[i])
print(len(gender_lst))

108


In [52]:
sex_dict = {}
for i in df.gender.unique():
  sex_dict.update({i: gender_lst.count(i)})

sex_dict = {k: v for k, v in sex_dict.items() if v}

print(sex_dict)

{'2: Female': 78, '1: Male': 26, '3: Other': 4}


In [54]:
sex_dict['Unknown'] = sex_dict[list(sex_dict.keys())[2]]
del sex_dict[list(sex_dict.keys())[3]]
print(sex_dict)

{'2: Female': 78, '1: Male': 26, '3: Other': 4}


In [55]:
diag_lst = []
for i in range(len(df.sweetgoals_id.values)):
  # print(i)
  if not np.isnan(df.diagnosis_duration.values[i]):
    diag_lst.append(df.diagnosis_duration.values[i])
print(len(diag_lst), diag_lst)

106 [10.6, 19.0, 4.1, 4.3, 10.8, 3.8, 5.0, 16.6, 7.2, 2.5, 16.4, 12.5, 4.6, 13.9, 9.4, 13.2, 8.7, 16.2, 13.9, 17.2, 2.9, 6.6, 4.5, 6.7, 13.3, 11.3, 11.3, 14.4, 9.8, 7.9, 17.2, 6.3, 3.9, 5.7, 10.3, 14.4, 14.3, 3.0, 13.0, 4.3, 4.7, 18.2, 15.1, 24.9, 9.1, 11.2, 4.5, 20.3, 9.9, 9.7, 20.5, 12.6, 4.7, 9.7, 18.1, 16.4, 10.4, 16.6, 8.4, 9.4, 11.9, 12.2, 8.9, 10.7, 10.0, 10.9, 7.7, 16.1, 10.5, 13.8, 16.6, 20.8, 7.7, 9.6, 14.4, 10.2, 3.2, 12.5, 10.3, 14.0, 13.4, 16.9, 4.7, 12.6, 17.0, 14.6, 8.8, 7.1, 13.0, 16.6, 21.2, 17.4, 27.0, 10.7, 4.8, 13.8, 24.2, 11.0, 2.4, 17.6, 15.0, 10.4, 2.9, 16.8, 12.0, 9.5]


In [56]:
print("avg and std:", round(np.mean(diag_lst), 2), round(np.std(diag_lst), 2))
print('diag duration range:', np.min(diag_lst), '-' , np.max(diag_lst))
print('The number of subjects does not have diagnosis duration:', 108 - len(diag_lst))

avg and std: 11.5 5.29
diag duration range: 2.4 - 27.0
The number of subjects does not have diagnosis duration: 2


In [57]:
a1c_lst = []
for i in range(len(df.sweetgoals_id.values)):
  # print(i)
  if not np.isnan(df.A1c.values[i]):
    a1c_lst.append(df.A1c.values[i])

print("The number of subjects have a1c values:", len(a1c_lst))

The number of subjects have a1c values: 94


In [58]:
print("avg and std:", round(np.mean(a1c_lst), 2), round(np.std(a1c_lst), 2))
print('a1c range:', np.min(a1c_lst), '-' , np.max(a1c_lst))
print('The number of subjects does not have A1c:', 108 - len(a1c_lst))

avg and std: 8.4 1.3
a1c range: 6.3 - 14.4
The number of subjects does not have A1c: 14


In [59]:
insurance_list = pd.Series(list(df.Insurance_0.values)).fillna(0).tolist()

insurance_dict = {}
for i in [0, 1, 2]:
  insurance_dict.update({i: insurance_list.count(i)})

# insurance_dict = {k: v for k, v in insurance_dict.items() if v}

print(insurance_dict)

{0: 12, 1: 78, 2: 18}


## CGM data

In [5]:
id = df.app_id.values
demo_file = './demographic.csv'
cgm_model_df = pd.read_csv(demo_file)[['app_id', 'cgm_model']]
# cgm_model_df.head(3)

In [ ]:
model_lst = []
for i in cgm_model_df.values:
  if i[0] in id:
    model_lst.append(i[1])
print(len(model_lst))

106


In [ ]:
model_dict = {}
for i in cgm_model_df.cgm_model.unique():
  model_dict.update({i: model_lst.count(i)})

model_dict = {k: v for k, v in model_dict.items() if v}

print(model_dict)

{nan: 2, 'Dexcom G6': 91, 'Dexcom': 7, 'Dexcom g6': 4, 'Guardian sensor (G3 Medtronic)': 1, 'Dexcom G5': 1}


In [ ]:
print('dexcom:', 91 + 7 + 4 + 1, '\nmedtronic:', 1, '\nunknown:', 108 - 106 + 2)

dexcom: 103 
medtronic: 1 
unknown: 4


In [ ]:
cgm_summary = './CGM_summary.csv'

cgm_df = pd.read_csv(cgm_summary)
cgm_df.head(3)

,Subject,Key,Start,End,Duration,RecordDays,MissingDays,ValidDays
0,619e52b1d497d2000fd8661f,1,2021-02-25,2021-11-23,272,261,11,258
1,6287b500f6ce3300102a91cf,2,2021-08-19,2022-05-19,274,241,33,219
2,61ef29c687d6c6000c7e426f,3,2021-04-18,2022-01-17,275,102,173,66


In [ ]:
print('total duration:', np.sum(cgm_df.Duration.values))

print('duration avg and std:', np.mean(cgm_df.Duration.values), np.std(cgm_df.Duration.values))
print('range:', np.min(cgm_df.Duration.values), '-', np.max(cgm_df.Duration.values))

print('total dyas without data record:', np.sum(cgm_df.MissingDays.values))
print('total dyas with more than 70% daily data record:', np.sum(cgm_df.ValidDays.values))

total duration: 118325
duration avg and std: 1095.601851851852 579.8944019422657
range: 272 - 2412
total dyas without data record: 32931
total dyas with more than 70% daily data record: 76435


In [ ]:
# for days with more than 70% record
cgm_df2 = pd.read_csv('./summary_TIR_meanBG_gluVar.csv')
cgm_df2.head(3)

,TIR,TAR,TBR,meanBG,GlucoseVar
0,0.652778,0.250000,0.097222,149.371528,0.318194
1,0.295139,0.704861,0.000000,211.121528,0.259810
2,0.180556,0.819444,0.000000,226.690972,0.191743


In [ ]:
meanBG = cgm_df2.meanBG.values

print('daily mean BG avg and std:', np.round(np.mean(meanBG), 2), np.round(np.std(meanBG), 2))
print('range:', np.round(np.min(meanBG), 2), '-', np.max(meanBG))

daily mean BG avg and std: 195.17 49.19
range: 70.35 - 401.0


In [ ]:
TIR = cgm_df2.TIR.values

print('daily TIR avg and std:', np.round(np.mean(TIR), 4), np.round(np.std(TIR), 4))
print('range:', np.min(TIR), '-', np.max(TIR))

daily TIR avg and std: 0.4697 0.2411
range: 0.0 - 1.0


# Other paper data

## High adherence ratio

In [6]:
df_valid = pd.read_csv('./yearly_adherence_trends.csv')
df_valid

In [ ]:
lst = list(df_valid.Valid_ratio.values)
print(min(df_valid.Valid_ratio.values), lst.index(min(df_valid.Valid_ratio.values)), df_valid.values[lst.index(min(df_valid.Valid_ratio.values))])
print(max(df_valid.Valid_ratio.values), lst.index(max(df_valid.Valid_ratio.values)), df_valid.values[lst.index(max(df_valid.Valid_ratio.values))])

0.562466307277628 196 [ 7.         16.          0.56246631]
0.7347129506008011 358 [12.         25.          0.73471295]
